<a href="https://colab.research.google.com/github/Feekah/HumanPoseEstimation/blob/main/Dissertation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [174]:
import mediapipe as mp
import cv2
import pandas as pd
import pickle
import numpy as np
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import mediapipe as mp
import cv2
import pandas as pd
import pickle
import numpy as np
import csv
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score, confusion_matrix, roc_curve, auc
from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score, confusion_matrix, roc_curve, auc

#Define Functions

In [175]:
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

def init_csv(dataset_path: str):
    '''
    Create a blank csv file with just columns
    '''

    # Write all the columns to a file
    with open(dataset_path, mode="w", newline="") as f:
        csv_writer = csv.writer(f, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(landmarks)

#define a function to calculate angles between joints
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle >180.0:
        angle = 360-angle

    return angle

def export_landmark_to_csv(dataset_path: str, results, position: str, angle, angle2, angle3, label: str) -> None:
    '''
    Export Labeled Data from detected landmark to csv
    '''
    landmarks = results.pose_landmarks.landmark
    keypoints = []

    try:
        # Extract coordinate of important landmarks
        for lm in IMPORTANT_LMS:
            keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
            keypoints.append([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])

        keypoints = list(np.array(keypoints).flatten())

        # Insert action as the label (first column)
        keypoints.insert(0, position)
        keypoints.insert(1, angle)
        keypoints.insert(2, angle2)
        keypoints.insert(3, angle3)
        keypoints.insert(4, label)

        # Append new row to .csv file
        with open(dataset_path, mode="a", newline="") as f:
            csv_writer = csv.writer(f, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)


    except Exception as e:
        print(e)
        pass

def describe_dataset(dataset):
    ''''''

    data = pd.read_csv(dataset)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Position: \n{data['Label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {duplicate.sum(axis=1)}")

    return data

def process_single_video(video, csv_path, video_label):
    cap = cv2.VideoCapture(video)
    init_csv(csv_path)

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, image = cap.read()
            if not ret:
                break

            # Reduce size of a frame
            image = rescale_frame(image, 50)
            image = cv2.flip(image, 1)

            # Recolor image from BGR to RGB for mediapipe
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            results = pose.process(image)

            # Recolor image from BGR to RGB for mediapipe
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            lms = results.pose_landmarks.landmark

            hip = [lms[mp_pose.PoseLandmark.LEFT_HIP.value].x,lms[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [lms[mp_pose.PoseLandmark.LEFT_KNEE.value].x,lms[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [lms[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,lms[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            shoulder = [lms[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,lms[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            foot_index = [lms[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,lms[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]

            # Calculate angle
            angle = calculate_angle(hip, knee, ankle)
            angle2 = calculate_angle(shoulder, hip, knee)
            angle3 = calculate_angle(knee, ankle, foot_index)

            # Draw landmarks and connections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            cv2.imshow("CV2", image)

            # Pressed key for action
            k = cv2.waitKey(1) & 0xFF

            if angle < 110:
                export_landmark_to_csv(csv_path, results, "down", angle, angle2, angle3, video_label)
            elif angle > 150:
                export_landmark_to_csv(csv_path, results, "up", angle, angle2, angle3, video_label)
            # Press q to stop
            elif k == ord("q"):
                break
            else: continue

        cap.release()
        cv2.destroyAllWindows()

        # (Optional)Fix bugs cannot close windows in MacOS (https://stackoverflow.com/questions/6116564/destroywindow-does-not-close-window-on-mac-using-python-and-opencv)
        for i in range (1, 5):
            cv2.waitKey(1)

def evaluate_algorithms(X_train, y_train, X_test, y_test, algorithms):
    # Initialize dictionaries to store metric results
    accuracy_results = {}
    precision_results = {}
    recall_results = {}
    f1_results = {}

    # Iterate through algorithms and evaluate performance
    for name, model in algorithms:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy_results[name] = accuracy_score(y_test, y_pred)
        precision_results[name] = precision_score(y_test, y_pred, average='weighted')
        recall_results[name] = recall_score(y_test, y_pred, average='weighted')
        f1_results[name] = f1_score(y_test, y_pred, average='weighted')

    # Print the results
    print("Algorithm\tAccuracy\tPrecision\tRecall\t\tF1-score")
    for name in algorithms:
        print(f"{name[0]}\t\t{accuracy_results[name[0]]:.4f}\t\t{precision_results[name[0]]:.4f}\t\t{recall_results[name[0]]:.4f}\t\t{f1_results[name[0]]:.4f}")



In [176]:
#Intialize the Mediapipe libraries
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

#Highlight the important joints for the squat exercise

# Determine important landmarks for squat
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "LEFT_HIP",
    "LEFT_KNEE",
    "LEFT_ANKLE",
    "LEFT_FOOT_INDEX"
]

# Generate all columns of the data frame

landmarks = ["Position", "Hip_Knee-Ankle_angle", "Shoulder_hip_knee_angle", "Knee_ankle_foot_angle", "Label"] # Label column

for lm in IMPORTANT_LMS:
    landmarks += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

video_labels = {
    "Correct_Squat_Female.mp4": "Correct Pose",
    "Correct_Squat_Female_2.mp4": "Correct Pose",
    "Correct_Squat_Male.mp4" : "Correct Pose",
    "Incorrect_Back_Female.mp4" : "Low back",
    "Incorrect_Back_Male.mp4" : "Low back",
    "Incorrect_Dorsiflexion_Female.mp4" : "Knee too forward",
    "Incorrect_Dorsiflexion_Male.mp4" : "Knee too forward",
    "Incorrect_Halfway_Female.mp4" : "Early stopping",
    "Incorrect_Halfway_Male.mp4": "Early stopping"

    # ... add other videos and their corresponding labels ...
}

videos = [
    "Correct_Squat_Female.mp4",
    "Correct_Squat_Female_2.mp4",
    "Correct_Squat_Male.mp4",
    "Incorrect_Back_Female.mp4",
    "Incorrect_Back_Male.mp4",
    "Incorrect_Dorsiflexion_Female.mp4",
    "Incorrect_Dorsiflexion_Male.mp4",
    "Incorrect_Halfway_Female.mp4",
    "Incorrect_Halfway_Male.mp4"
    # ... add paths to your other videos here ...
]

output_csv = "Documents/PEVideos/csvfiles/concat3.csv"

for video in videos:
    csv_filename = f"{video.split('.')[0]}.csv"
    label = video_labels.get(video, "Unknown Pose")
    process_single_video(video, csv_filename, label)

# Get a list of all generated CSV files
csv_files = [f"{video.split('.')[0]}.csv" for video in videos]

# Concatenate CSV files with the same headers
concatenated_data = pd.concat([pd.read_csv(csv_file) for csv_file in csv_files], ignore_index=True)
concatenated_data.to_csv(output_csv, index=False)

describe_dataset(output_csv)




Headers: ['Position', 'Hip_Knee-Ankle_angle', 'Shoulder_hip_knee_angle', 'Knee_ankle_foot_angle', 'Label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'left_foot_index_x', 'left_foot_index_y', 'left_foot_index_z', 'left_foot_index_v']
Number of rows: 4217 
Number of columns: 29

Position: 
Correct Pose        1591
Low back             962
Knee too forward     877
Early stopping       787
Name: Label, dtype: int64

Missing values: False

Duplicate Rows : Series([], dtype: float64)


,Position,Hip_Knee-Ankle_angle,Shoulder_hip_knee_angle,Knee_ankle_foot_angle,Label,nose_x,nose_y,nose_z,nose_v,left_shoulder_x,...,left_knee_z,left_knee_v,left_ankle_x,left_ankle_y,left_ankle_z,left_ankle_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v
0,up,174.698598,172.343197,93.278529,Correct Pose,0.574678,0.150211,-0.008606,0.999822,0.464209,...,0.449507,0.286480,0.468858,0.752013,0.724832,0.569669,0.583139,0.759160,0.675724,0.754430
1,up,174.507930,172.883083,90.915794,Correct Pose,0.574065,0.150198,-0.017245,0.999812,0.464392,...,0.366694,0.292429,0.468090,0.757568,0.580367,0.571735,0.583615,0.760920,0.505526,0.756755
2,up,173.967075,172.793623,89.291976,Correct Pose,0.571460,0.150073,-0.019840,0.999802,0.466042,...,0.369018,0.295480,0.467954,0.760369,0.589789,0.571601,0.584569,0.761555,0.522359,0.757682
3,up,174.659826,174.412664,88.035739,Correct Pose,0.567895,0.149532,-0.028426,0.999796,0.469075,...,0.352686,0.301047,0.468135,0.763055,0.563281,0.573378,0.587912,0.762573,0.486747,0.758827
4,up,174.742855,175.284304,87.389848,Correct Pose,0.564277,0.148921,-0.035028,0.999792,0.470913,...,0.336142,0.306669,0.468568,0.763610,0.537968,0.575128,0.590836,0.762646,0.458257,0.759836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4212,up,167.846551,170.942405,88.922936,Early stopping,0.542405,0.169335,-0.132593,0.998226,0.471843,...,0.416010,0.345079,0.500919,0.715793,0.688242,0.614774,0.606811,0.729999,0.596042,0.777680
4213,up,168.267903,170.359673,89.689894,Early stopping,0.542545,0.169321,-0.127965,0.998202,0.472086,...,0.415456,0.340774,0.500943,0.717501,0.687317,0.606582,0.606845,0.733109,0.593732,0.770848
4214,up,168.953116,169.851681,92.015083,Early stopping,0.542642,0.169348,-0.157740,0.998248,0.474048,...,0.418751,0.335773,0.500952,0.719273,0.700607,0.596960,0.606443,0.739156,0.604786,0.762983
4215,up,168.859261,169.635093,92.711632,Early stopping,0.542947,0.169306,-0.192699,0.998327,0.474692,...,0.392808,0.330188,0.501701,0.719466,0.652886,0.585705,0.606482,0.740831,0.566388,0.752264


#Model Training

In [149]:
# Define algorithms
algorithms = [
    #("LR", LogisticRegression(max_iter=1000)),
    ("SVC", SVC(probability=True)),
    ('KNN', KNeighborsClassifier()),
    ("DTC", DecisionTreeClassifier()),
    ("SGDC", CalibratedClassifierCV(SGDClassifier())),
    ("NB", GaussianNB()),
    ('RF', RandomForestClassifier()),
]

In [159]:
data = pd.read_csv("concat2.csv")

#Using Label - Encoding

In [165]:
le = data.copy()

# Separate categorical columns
categorical_columns = ['Position', 'Label']  # Add more columns if needed

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to each categorical column
for col in categorical_columns:
    le[col] = label_encoder.fit_transform(le[col])

# Extract features and class
X = le.drop("Label", axis=1) # features
y = le["Label"]

# Split train set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)
y_test.head(3)


4062    1
3064    2
292     0
Name: Label, dtype: int64

In [161]:
# Call the function
evaluate_algorithms(X_train, y_train, X_test, y_test, algorithms)

Algorithm	Accuracy	Precision	Recall		F1-score
SVC		0.5616		0.7131		0.5616		0.5357
KNN		0.7938		0.7977		0.7938		0.7937
DTC		0.9858		0.9858		0.9858		0.9858
SGDC		0.5450		0.6636		0.5450		0.5200
NB		0.8096		0.8172		0.8096		0.8110
RF		1.0000		1.0000		1.0000		1.0000


#Using One-Hot Encoding

In [162]:
oh = data.copy()
# Separate categorical columns
categorical_columns = ['Position']  # Add more columns if needed

# Apply one-hot encoding to categorical columns
df_encoded = pd.get_dummies(oh, columns=categorical_columns)

df_encoded['Label'] = label_encoder.fit_transform(df_encoded['Label'])

df_encoded.head()

# Extract features and class
X = le.drop("Label", axis=1) # features
y = le["Label"]

# Split train set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
y_test.head(3)

4062    1
3064    2
292     0
Name: Label, dtype: int64

In [163]:
# Call the function
evaluate_algorithms(X_train, y_train, X_test, y_test, algorithms)

Algorithm	Accuracy	Precision	Recall		F1-score
SVC		0.5770		0.7210		0.5770		0.5521
KNN		0.8092		0.8157		0.8092		0.8097
DTC		0.9858		0.9858		0.9858		0.9858
SGDC		0.5533		0.6915		0.5533		0.5132
NB		0.7938		0.7977		0.7938		0.7946
RF		1.0000		1.0000		1.0000		1.0000


In [167]:
model1 = DecisionTreeClassifier().fit(X, y)
        #y_pred = model.predict(X_test)

In [ ]:
cap = cv2.VideoCapture(0)

init_csv(csv_path)

squat_counter = 0
squat_stage = 'Up'

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
      ret, image = cap.read()
      # Recolor image to RGB
      image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      image.flags.writeable = False

      # Make detection
      results = pose.process(image)

      # Recolor back to BGR
      image.flags.writeable = True
      image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

      # Extract landmarks
      try:


        lms = results.pose_landmarks.landmark

        hip = [lms[mp_pose.PoseLandmark.LEFT_HIP.value].x,lms[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        knee = [lms[mp_pose.PoseLandmark.LEFT_KNEE.value].x,lms[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        ankle = [lms[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,lms[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
        shoulder = [lms[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,lms[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        foot_index = [lms[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,lms[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]

        # Calculate angle
        angle = calculate_angle(hip, knee, ankle)
        angle2 = calculate_angle(shoulder, hip, knee)
        angle3 = calculate_angle(knee, ankle, foot_index)

        # Draw landmarks and connections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # Display the saved count
        cv2.putText(image, f"UP saved: {squat_counter}", (50, 150), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(image, f"DOWN saved: {squat_counter}", (50, 200), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 1, cv2.LINE_AA)

        cv2.imshow("CV2", image)

        if angle < 150 and squat_stage == 'Up':
          export_landmark_to_csv(csv_path, results, "down", angle, angle2, angle3, video_label)
          squat_stage == 'Down'
        elif angle > 70 and squat_stage == 'Down':
          export_landmark_to_csv(csv_path, results, "up", angle, angle2, angle3, video_label)
          squat_stage == 'Up'
          squat_counter +=1

            # Press q to stop
        elif k == ord("q"):
          break
        else: continue

      except:
            pass
